In [1]:
import numpy as np
import src.models.helpers as mh
import plotly.graph_objects as go
import src.modelling_fev1.uecfev1 as uecfev1

In [38]:
ecFEV1 = mh.VariableNode("ecFEV1 (L)", 0, 6, 0.05, prior={"type": "uniform"})
uecFEV1 = mh.VariableNode("Underlying ecFEV1 (L)", 0, 6, 0.05, prior=None)
std_gauss = 0.3

# Plot the PDF of ecFEV1 given the middle bin of uecFEV1
uecfev1_bin = uecFEV1.get_bins_arr()[uecFEV1.card // 2]
print("uecFEV1 bin", uecfev1_bin)
# Get the PDF
pdf = np.zeros(ecFEV1.card)
for i, z in enumerate(ecFEV1.midbins):
    pdf[i] = uecfev1.PDF_conv_uni_gausian(z, uecfev1_bin[0], uecfev1_bin[1], std_gauss)
# Norm the pdf
pdf /= np.sum(pdf)
# Plot pdf with graph objects library
fig = go.Figure()
fig.add_trace(go.Bar(x=ecFEV1.midbins, y=pdf))
title = f"P(ecFEV1 | uecFEV1={uecfev1_bin} L)"
fig.update_layout(title=title, height=300, width=650)
fig.update_xaxes(title_text=ecFEV1.name)
fig.show()

uecFEV1 bin [3.   3.05]


In [39]:
ecFEV1 = mh.VariableNode("ecFEV1 (L)", 0, 6, 0.05, prior={"type": "uniform"})
uecFEV1 = mh.VariableNode("Underlying ecFEV1 (L)", 0, 6, 0.05, prior=None)
std_gauss = 0.23

# Select bin that's no troubled by borders
uecfev1_bin = uecFEV1.get_bins_arr()[uecFEV1.card // 2]
print("uecFEV1 bin", uecfev1_bin)
# Get the PDF
pdf = np.zeros(ecFEV1.card)
for i, z in enumerate(ecFEV1.midbins):
    pdf[i] = uecfev1.PDF_conv_uni_gausian(z, uecfev1_bin[0], uecfev1_bin[1], std_gauss)
# Norm the pdf
pdf /= np.sum(pdf)

# The same PDF will be shifted across all uecFEV1/ ecFEV1 pairs
# When hitting a border, the PDF will be truncated
cpt = np.zeros((ecFEV1.card, uecFEV1.card))
pdf_peek_idx = uecFEV1.card // 2
for uecFEV1_idx, uecfev1_bin in enumerate(uecFEV1.get_bins_arr()):
    pdf_trunc = np.zeros(len(pdf))
    ecFEV1_idx_peek = uecFEV1_idx
    peek_diff = pdf_peek_idx - ecFEV1_idx_peek
    if peek_diff == 0:
        pdf_trunc = pdf
    elif peek_diff > 0:
        pdf_trunc[0:-peek_diff] = pdf[peek_diff:]
    else:
        pdf_trunc[-peek_diff:] = pdf[:peek_diff]
    # Norm the pdf
    pdf_trunc /= np.sum(pdf_trunc)
    cpt[:, uecFEV1_idx] = pdf_trunc

uecFEV1 bin [3.   3.05]


In [50]:
import src.models.cpts.helpers as cpth
import src.data.helpers as dh

fig, title = cpth.plot_2d_cpt(cpt, ecFEV1, uecFEV1, height=6000, y_label_two_lines=True, p_range=[0, 0.16], vspace=0.002, invert=False)
title = title + f" for an ecFEV1 variability of {std_gauss}"
fig.update_layout(title=title)

fig.write_image(f"{dh.get_path_to_main()}PlotsBreathe/CPTs/{title}.pdf")

In [ ]:
# NOTE: THIS IMPLEMENTATION TAKES TOO LONG TO RUN, WHEN WITH ANALYTICAL SOLUTION INSTEAD
def get_noise_model_CPT(
    pVar: mh.VariableNode,
    cVar: mh.VariableNode,
    c_resolution: int,
    std_gauss: float,
    n: int,
    debug=False,
):
    """
    pVar: parent variable, corresponds to the true value that would be obtained for that variable if we had a magic measurement devive
    cVar: child variable, corresponds to the measured value, or a value that could be obtained with a measurement device from clinical practice
    c_resolution: number of decimals, correspond to the resolution of the measurement device
    std_gauss: standard deviation of the gaussian noise of the measurement device
    n: number of samples to generate per bin of the parent variable
    """
    cpt = np.zeros((cVar.card, pVar.card))
    # Do it for one bin that's central,
    # Then shift the values to the other bins

    middle_bin = pVar.get_bins_arr()[pVar.card // 2]
    real_arr = pVar.sample_from_bin(middle_bin, n)
    # Add gaussian noise
    # Truncnorm ensures that sampled values are plausible (within the possible values of the child variable)
    # noisy_real_arr = [truncnorm(cVar.a, cVar.b, loc, std_gauss).rvs(1) for loc in real_arr]
    noisy_real_arr = np.random.normal(real_arr, std_gauss)
    # Round to measurement resolution
    rounded_arr = np.round(noisy_real_arr, c_resolution)
    # Discard values that are out of the possible values of the child variable
    rounded_arr = rounded_arr[(rounded_arr >= cVar.a) & (rounded_arr <= cVar.b)]
    # bin up the rounded values to the child variable
    p_cVar_given_pVar_obs = cVar.bin_up(rounded_arr, normalise=True)

    # # Get
    # # Add to the CPT
    # cpt[:, pVar.card // 2] = p_cVar_given_pVar_obs

    # assert np.isclose(
    #     cpt[:, i].sum(), 1, atol=cVar.tol
    # ), f"The sum of probabilities should be 1, got {cpt[:, i].sum()} while calculating P({cVar.name}|{pVar.name}={pVar_bin})"
    return p_cVar_given_pVar_obs

ecFEV1 = mh.VariableNode("ecFEV1 (L)", 0, 6, 0.05, prior={"type": "uniform"})
uecFEV1 = mh.VariableNode("Underlying ecFEV1 (L)", 0, 6, 0.05, prior=None)
std_gauss = 0.3
n_decimals = 2
n = 100000000

p = get_noise_model_CPT(uecFEV1, ecFEV1, n_decimals, std_gauss, n, debug=False)